This simulator would in theory take the prediction of the tape data and 

I have a few ideas:
using the same momentum strategy as last time, basically by looking ahead of the prediction to decide if should buy or sell
for the merged lob, we also check the next timestamp to see if say it gets to 10s, so we place our bid or ask
the third idea is to merge our prediction with the lob data, and then count the number of matches


In [40]:
import pandas as pd

# Load the datasets
tape_data = pd.read_csv('../predicted_tape.csv')
lob_data = pd.read_csv('../merge/merged_lob.csv')

# Display the first few rows of each dataset to understand their structure
tape_data.head(), lob_data.head()


(             timestamp  transaction_price
 0  2025-01-02 00:00:10              267.0
 1  2025-01-02 00:00:11              268.0
 2  2025-01-02 00:00:12              270.0
 3  2025-01-02 00:00:13              267.0
 4  2025-01-02 00:00:14              267.0,
    Timestamp       Bid         Ask
 0      0.000        []          []
 1      0.279  [[1, 6]]          []
 2      1.333  [[1, 6]]  [[800, 1]]
 3      1.581  [[1, 6]]  [[799, 1]]
 4      1.643  [[1, 6]]  [[798, 1]])

In [41]:
# Convert 'timestamp' in tape_data to seconds since the start of the day
tape_data['timestamp'] = pd.to_datetime(tape_data['timestamp'])
tape_data['timestamp_seconds'] = tape_data['timestamp'].dt.hour * 3600 + tape_data['timestamp'].dt.minute * 60 + tape_data['timestamp'].dt.second

# Convert 'Timestamp' in lob_data to an absolute second count from the start of the day, assuming it starts from zero
# lob_data['Timestamp'] = lob_data['Timestamp'].round().astype(int)  # Assuming the timestamps in lob_data are in whole seconds

# Display the modified data for verification
tape_data.head(), lob_data.head(10)


(            timestamp  transaction_price  timestamp_seconds
 0 2025-01-02 00:00:10              267.0                 10
 1 2025-01-02 00:00:11              268.0                 11
 2 2025-01-02 00:00:12              270.0                 12
 3 2025-01-02 00:00:13              267.0                 13
 4 2025-01-02 00:00:14              267.0                 14,
    Timestamp                   Bid                             Ask
 0      0.000                    []                              []
 1      0.279              [[1, 6]]                              []
 2      1.333              [[1, 6]]                      [[800, 1]]
 3      1.581              [[1, 6]]                      [[799, 1]]
 4      1.643              [[1, 6]]                      [[798, 1]]
 5      1.736    [[261, 1], [1, 6]]                      [[798, 1]]
 6      1.984    [[261, 1], [1, 6]]                      [[797, 1]]
 7      2.015    [[261, 1], [1, 6]]            [[338, 3], [797, 1]]
 8      2.139    [[26

In [12]:
# Merge tape_data and lob_data based on the closest matching timestamps
# We'll use the merge_asof method which is designed to merge on nearest key rather than exact matches

# Ensure both dataframes are sorted by their respective time columns
tape_data = tape_data.sort_values(by='timestamp_seconds')
lob_data = lob_data.sort_values(by='Timestamp')

tape_data['timestamp_seconds'] = tape_data['timestamp_seconds'].astype('int64')
# Perform an asof merge, looking for the nearest previous or equal timestamp in lob_data for each entry in tape_data
merged_data = pd.merge_asof(tape_data, lob_data, left_on='timestamp_seconds', right_on='Timestamp')

# Display the merged data to verify correctness
merged_data.head()


,timestamp,transaction_price,timestamp_seconds,Timestamp,Bid,Ask
0,2025-01-02 00:00:10,267.0,10,10,"[[267, 6], [261, 1], [260, 11], [259, 1], [178...","[[270, 1], [271, 5], [272, 1], [275, 3], [277,..."
1,2025-01-02 00:00:11,268.0,11,11,"[[267, 5], [261, 1], [260, 11], [259, 1], [193...","[[269, 1], [271, 5], [275, 8], [277, 2], [281,..."
2,2025-01-02 00:00:12,270.0,12,12,"[[267, 3], [261, 1], [260, 11], [259, 1], [193...","[[270, 5], [273, 3], [279, 5], [281, 6], [308,..."
3,2025-01-02 00:00:13,267.0,13,13,"[[267, 3], [264, 1], [261, 1], [260, 11], [193...","[[270, 3], [273, 3], [279, 5], [280, 4], [281,..."
4,2025-01-02 00:00:14,267.0,14,14,"[[264, 1], [261, 1], [260, 11], [193, 15], [18...","[[273, 3], [278, 4], [279, 5], [282, 2], [744,..."


In [55]:
import ast  # To safely evaluate string representation of lists
# 
# def momentum_trading_revised(data, initial_capital):
#     capital = initial_capital
#     position = 0  # No position initially
# 
#     for i in range(len(data) - 1):
#         current_price = data['transaction_price'].iloc[i]
#         next_price = data['transaction_price'].iloc[i + 1]
# 
#         # Parse bid and ask lists from the current row
#         current_bids = ast.literal_eval(data['Bid'].iloc[i])
#         current_asks = ast.literal_eval(data['Ask'].iloc[i])
# 
#         # Determine the best bid and ask prices and volumes
#         if current_bids:
#             best_bid = max(current_bids, key=lambda x: x[0])  # Highest bid price
#             best_bid_price, best_bid_volume = best_bid
#         else:
#             best_bid_price, best_bid_volume = 0, 0
#         
#         if current_asks:
#             best_ask = min(current_asks, key=lambda x: x[0])  # Lowest ask price
#             best_ask_price, best_ask_volume = best_ask
#         else:
#             best_ask_price, best_ask_volume = float('inf'), 0
# 
#         # Buy if next price is higher than current price and there are available asks
#         if next_price > current_price and next_price > best_ask_price:
#             # max_affordable_volume = capital / best_ask_price
#             buy_volume = 1 # min(max_affordable_volume, best_ask_volume)
#             capital -= buy_volume * best_ask_price
#             position += buy_volume
# 
#         # Sell if next price is lower than current price, there are available bids, and position > 0
#         elif next_price < current_price and position > 0 and best_bid_price > next_price:
#             sell_volume = 1 # min(position, best_bid_volume)
#             capital += sell_volume * best_bid_price
#             position -= sell_volume
# 
#     # Final sell at the last available bid price to liquidate position
#     if position > 0 and ast.literal_eval(data['Bid'].iloc[-1]):
#         final_bid = max(ast.literal_eval(data['Bid'].iloc[-1]), key=lambda x: x[0])
#         final_price = final_bid[0]
#         capital += position * final_price
# 
#     return capital
# 
# # Initial capital to start with
# initial_capital = 10000  # For example, $10,000
# 
# # Run the trading simulator
# final_capital = momentum_trading_revised(merged_data, initial_capital)
# final_capital
def momentum_trading_with_pointers(tape_data, lob_data, initial_capital):
    capital = initial_capital
    position = 0  # No position initially
    
    tape_index = 0
    lob_index = 0
    tape_len = len(tape_data)
    lob_len = len(lob_data)

    while tape_index < tape_len - 1 and lob_index < lob_len:
        # Current positions in both datasets
        tape_ts = tape_data['timestamp_seconds'].iloc[tape_index]
        lob_ts = lob_data['Timestamp'].iloc[lob_index]

        # Move LOB pointer to match or exceed tape timestamp
        # while lob_index < lob_len - 1 and lob_data['Timestamp'].iloc[lob_index + 1] <= tape_ts:
        #     lob_index += 1
        if lob_index < lob_len - 1 and lob_ts <= tape_ts:
            lob_index += 1
        elif tape_data['timestamp_seconds'].iloc[tape_index + 1] > lob_data['Timestamp'].iloc[lob_index] > tape_ts:
            
        # If timestamps match, execute trading logic
        # if lob_ts == tape_ts:
            current_price = tape_data['transaction_price'].iloc[tape_index]
            next_price = tape_data['transaction_price'].iloc[tape_index + 1] if tape_index + 1 < tape_len else current_price
    
            # Parse bid and ask lists
            current_bids = ast.literal_eval(lob_data['Bid'].iloc[lob_index])
            current_asks = ast.literal_eval(lob_data['Ask'].iloc[lob_index])
    
            # Determine the best bid and ask
            if current_bids:
                best_bid = max(current_bids, key=lambda x: x[0])
                best_bid_price, best_bid_volume = best_bid
            else:
                best_bid_price, best_bid_volume = 0, 0
            
            if current_asks:
                best_ask = min(current_asks, key=lambda x: x[0])
                best_ask_price, best_ask_volume = best_ask
            else:
                best_ask_price, best_ask_volume = float('inf'), 0
    
            # Buy if predicted next price is higher than current and best ask
            if next_price > current_price and next_price > best_ask_price:
                buy_volume = 1  # Simplified to always trade one unit
                capital -= buy_volume * best_ask_price
                position += buy_volume
    
            # Sell if predicted next price is lower than current and best bid
            elif next_price < current_price and position > 0 and best_bid_price > next_price:
                sell_volume = 1  # Simplified to always trade one unit
                capital += sell_volume * best_bid_price
                position -= sell_volume
                
            lob_index += 1
        else:
            tape_index += 1
    print("shabi")
    # Final sell off all positions at last known bid price
    if position > 0 and ast.literal_eval(lob_data['Bid'].iloc[lob_index]):
        final_bid = max(ast.literal_eval(lob_data['Bid'].iloc[lob_index]), key=lambda x: x[0])
        final_price = final_bid[0]
        capital += position * final_price

    return capital

initial_capital = 10000  # For example, $10,000

# Run the improved trading simulator with the double pointer approach
final_capital = momentum_trading_with_pointers(tape_data, lob_data, initial_capital)
final_capital


shabi


99087